In [ ]:
#!pip install openai==1.29.0
import time
from tokencost import calculate_prompt_cost, calculate_completion_cost
from constants import *
from utils import *
from tqdm.auto import tqdm
import random
import json
# test constant import

from openai import AzureOpenAI
MODEL = "gpt-4-turbo-2024-04-09"
REGION = "eastus2"
API_KEY = "YOUR_API_KEY"
API_BASE = "https://api.openai.com"
ENDPOINT = f"{API_BASE}/{REGION}"

client = AzureOpenAI(
    api_key=API_KEY,
    api_version="2024-02-01",
    azure_endpoint=ENDPOINT,
)


In [ ]:
def chat(client, model, content, messages, role="user"):
    messages.append({"role": role, "content": content})
    prompt_cost = calculate_prompt_cost(messages, "gpt-4-turbo")
    completion = client.chat.completions.create(
        model=model,
        max_tokens=30,
        messages=messages
    )
    chat_response = completion
    answer = chat_response.choices[0].message.content
    completion_cost = calculate_completion_cost(answer, "gpt-4-turbo")
    print(f'ChatGPT: {answer}')
    messages.append({"role": "assistant", "content": answer})
    return answer, messages, prompt_cost + completion_cost

In [ ]:
data_dir = "data/"
train_file = f"{data_dir}train.json"

total_cost = 0
false = 0


with open(train_file, "r") as f:
    lines = f.readlines()
    test_files = [json.loads(line) for line in lines]
    # shuffle
    random.shuffle(test_files)

    save_dict = {}
    scene_list = {}

    for n, test_file in enumerate(tqdm(test_files[:200])):

        with open(test_file["text"], 'r') as f:
            text_log = f.readlines()
        
        scene = test_file["scene"]

        # Naive sampling code. You can replace it with your own sampling code to test different combinations.
        with open(test_files[n+1]["text"], 'r') as f:
            text_log_1 = f.readlines()
        preference_1 = test_files[n+1]["preference"]
        
        with open(test_files[n+2]["text"], 'r') as f:
            text_log_2 = f.readlines()
        preference_2 = test_files[n+2]["preference"]
            
        text_log_3, preference_3 = get_same_demo_text(test_file["preference"], test_files)
        # text_log_3, preference_3 = text_log, test_file["preference"]
        
        # shuffle the text logs and corresponding preferences
        text_logs = [[text_log_1, preference_1], [text_log_2, preference_2], [text_log_3, preference_3]]
        random.shuffle(text_logs)
        text_log_1, preference_1 = text_logs[0]
        text_log_2, preference_2 = text_logs[1]
        text_log_3, preference_3 = text_logs[2]
        
        messages = []

        instructions = "You are a robot assistant that can help summarize the host's preference."
        possible_preferences = Rearrangement[0]['Level0'] + Rearrangement[2]['Level2']
        # possible_preferences = Sequence_Preferences['name']

        instructions += f"Choose from following preference: \n{parse_concat(possible_preferences, replace=', ')}.\n"
        instructions += f"Text log file: \n {parse_concat(text_log_1, replace=', ')}.\n"
        instructions += f"Preference: {preference_1}.\n"
        instructions += f"Text log file: \n {parse_concat(text_log_2, replace=', ')}.\n"
        instructions += f"Preference: {preference_2}.\n"
        instructions += f"Text log file: \n {parse_concat(text_log_3, replace=', ')}.\n"
        instructions += f"Preference: {preference_3}.\n"
        instructions += f"Text log file: \n {parse_concat(text_log, replace=', ')}.\n"
        instructions += f"The user's preference is "

        # print(instructions)
        while(True):
            try:
                answer, messages, cost = chat(client, MODEL, instructions, messages, role="assistant")
                break
            except:
                time.sleep(1)

        total_cost += cost

        gt = test_file["preference"]

        answer = answer.lower().split(",")[0]

        if scene not in scene_list:
            scene_list[scene] = []

        if not compare(answer, gt, in_sequence=True):
            false += 1
            scene_list[scene].append(-2)
        else:
            scene_list[scene].append(2)

            
        print(f"True: {n+1-false}/{n+1}, total cost: {total_cost}")
        
    print(f"True: {n+1-false}/{len(test_files)}")

    for scene in scene_list:
        print(f"{scene_list[scene]}")


In [ ]:
for scene in scene_list:
    print(str(scene_list[scene]).replace(", ", " ")[1:-1])